In [2]:
from PIL import Image

images = [
    Image.open("/Users/d.rudenko/PycharmProjects/opensource/fastembed/tests/misc/image.jpeg"),
    Image.open(
        "/Users/d.rudenko/PycharmProjects/opensource/fastembed/tests/misc/small_image.jpeg"
    ),
]

In [ ]:
from fastembed.image.image_embedding import ColpaliImageModel

model_f_i = ColpaliImageModel(model_name="akshayballal/colpali-v1.2-merged")
fastembed_i_embeddings = list(model_f_i.embed(images))

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
from colpali_engine.models import ColPaliProcessor

model_name = "vidore/colpali-v1.2-merged"

processor = ColPaliProcessor.from_pretrained(model_name)
# Process the inputs
batch_images_onnx = processor.process_images(images)

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


In [4]:
import onnxruntime as ort

sess = ort.InferenceSession("/Users/d.rudenko/dev/qdrant/colpali-v1.2-merged-onnx/model.onnx")
image_embeddings_onnx = sess.run(
    [sess.get_outputs()[0].name],
    {
        "input_ids": batch_images_onnx["input_ids"].numpy(),
        "pixel_values": batch_images_onnx["pixel_values"].numpy(),
        "attention_mask": batch_images_onnx["attention_mask"].numpy(),
    },
)

In [8]:
import numpy as np

np.array(image_embeddings_onnx).shape

(1, 2, 1030, 128)

In [18]:
print(np.round(image_embeddings_onnx[0][:, :7, :7], decimals=3))

[[[ 0.015  0.051  0.059  0.026 -0.061 -0.027 -0.014]
  [-0.22  -0.111  0.046  0.081 -0.048 -0.052 -0.086]
  [-0.184 -0.131  0.004  0.062 -0.038 -0.059 -0.127]
  [-0.209 -0.113  0.015  0.059 -0.035 -0.035 -0.072]
  [-0.031 -0.044  0.092 -0.005  0.006 -0.057 -0.061]
  [-0.18  -0.039  0.031  0.003  0.083 -0.041  0.088]
  [-0.091  0.023  0.116 -0.02   0.039 -0.064 -0.026]]

 [[-0.25  -0.112 -0.065 -0.014  0.005 -0.092  0.024]
  [-0.22  -0.096 -0.014  0.039 -0.02  -0.12  -0.004]
  [-0.228 -0.114  0.031  0.019  0.034 -0.052 -0.031]
  [-0.274 -0.186  0.095 -0.019  0.017  0.021 -0.016]
  [-0.186 -0.061 -0.01   0.065 -0.058 -0.05   0.019]
  [-0.183 -0.11  -0.034 -0.042  0.026 -0.071  0.02 ]
  [-0.153 -0.072 -0.015  0.088 -0.081 -0.043  0.04 ]]]


In [13]:
np.array(image_embeddings_onnx)[0][0][0]

array([ 0.01533 ,  0.05118 ,  0.05948 ,  0.02583 , -0.06128 , -0.02682 ,
       -0.013565,  0.10254 , -0.0983  ,  0.1109  , -0.00342 , -0.0344  ,
       -0.00887 , -0.1616  ,  0.09814 ,  0.2257  ,  0.03976 ,  0.03687 ,
        0.1648  ,  0.06866 ,  0.0396  ,  0.1672  ,  0.1455  , -0.1387  ,
        0.1203  ,  0.04907 , -0.07965 , -0.0885  ,  0.01982 ,  0.0404  ,
       -0.07513 , -0.02844 ,  0.04337 ,  0.03857 , -0.1065  ,  0.0288  ,
       -0.1279  , -0.1126  ,  0.03363 , -0.0507  ,  0.11584 ,  0.0483  ,
        0.035   , -0.08417 , -0.0907  ,  0.0279  ,  0.1394  , -0.10364 ,
       -0.1471  , -0.07135 , -0.136   ,  0.1289  ,  0.082   ,  0.02232 ,
       -0.00571 , -0.02547 ,  0.1053  ,  0.0377  ,  0.0148  ,  0.02795 ,
       -0.01859 , -0.11066 , -0.12195 ,  0.0583  ,  0.0995  ,  0.01086 ,
        0.0859  ,  0.1302  , -0.10126 ,  0.005417,  0.05423 , -0.1808  ,
        0.1444  ,  0.1885  ,  0.09247 , -0.04718 ,  0.1018  , -0.02997 ,
       -0.0598  , -0.011284,  0.1203  , -0.1313  , 

In [5]:
import numpy as np

np.allclose(image_embeddings_onnx[0][0], fastembed_i_embeddings[0])

True

In [6]:
image_embeddings_onnx[0][0][0]

array([ 0.01533 ,  0.05118 ,  0.05948 ,  0.02583 , -0.06128 , -0.02682 ,
       -0.013565,  0.10254 , -0.0983  ,  0.1109  , -0.00342 , -0.0344  ,
       -0.00887 , -0.1616  ,  0.09814 ,  0.2257  ,  0.03976 ,  0.03687 ,
        0.1648  ,  0.06866 ,  0.0396  ,  0.1672  ,  0.1455  , -0.1387  ,
        0.1203  ,  0.04907 , -0.07965 , -0.0885  ,  0.01982 ,  0.0404  ,
       -0.07513 , -0.02844 ,  0.04337 ,  0.03857 , -0.1065  ,  0.0288  ,
       -0.1279  , -0.1126  ,  0.03363 , -0.0507  ,  0.11584 ,  0.0483  ,
        0.035   , -0.08417 , -0.0907  ,  0.0279  ,  0.1394  , -0.10364 ,
       -0.1471  , -0.07135 , -0.136   ,  0.1289  ,  0.082   ,  0.02232 ,
       -0.00571 , -0.02547 ,  0.1053  ,  0.0377  ,  0.0148  ,  0.02795 ,
       -0.01859 , -0.11066 , -0.12195 ,  0.0583  ,  0.0995  ,  0.01086 ,
        0.0859  ,  0.1302  , -0.10126 ,  0.005417,  0.05423 , -0.1808  ,
        0.1444  ,  0.1885  ,  0.09247 , -0.04718 ,  0.1018  , -0.02997 ,
       -0.0598  , -0.011284,  0.1203  , -0.1313  , 

In [7]:
fastembed_i_embeddings

[array([[ 0.01532745,  0.05117798,  0.05947876, ..., -0.12255859,
         -0.04510498, -0.00230598],
        [-0.22009277, -0.11071777,  0.04562378, ...,  0.00257111,
         -0.06988525,  0.12384033],
        [-0.18371582, -0.13085938,  0.00393677, ..., -0.02949524,
         -0.05444336,  0.1295166 ],
        ...,
        [-0.1418457 ,  0.01023102,  0.1239624 , ..., -0.00460434,
          0.17321777,  0.09454346],
        [-0.24572754, -0.06878662,  0.11834717, ..., -0.02763367,
         -0.03022766,  0.08917236],
        [-0.2211914 , -0.04171753,  0.19519043, ..., -0.01535797,
         -0.02432251, -0.03561401]], dtype=float32),
 array([[-2.49877930e-01, -1.11511230e-01, -6.51855469e-02, ...,
          3.19519043e-02,  3.44543457e-02, -1.33666992e-02],
        [-2.20336914e-01, -9.56420898e-02, -1.39694214e-02, ...,
         -8.88705254e-05, -1.57318115e-02, -1.00555420e-02],
        [-2.28271484e-01, -1.14501953e-01,  3.10058594e-02, ...,
          7.59277344e-02, -4.28466797e-02